In [ ]:
# 设置 Kaggle Notebook 启用自动代码补全
%config Completer.use_jedi = False

In [ ]:
import os
import torch

import numpy as np
import pandas as pd
import torch

import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

In [ ]:
# Device configuration
print(f"GPU available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Data

In [ ]:
# Load data from .csv by pandas
base_path = '/kaggle/input/digit-recognizer'
train_data = pd.read_csv(os.path.join(base_path, 'train.csv'))
submit_data = pd.read_csv(os.path.join(base_path, 'test.csv'))

In [ ]:
# Transforms to torch.Tensor
train_data = torch.tensor(train_data.values)
submit_data = torch.tensor(submit_data.values)

In [ ]:
print(train_data.shape, submit_data.shape)

In [ ]:
# Split training data to: 80% for training; 20% for testing;
train_size = int(0.8 * len(train_data))
train_features, train_labels = train_data[:train_size, 1:], train_data[:train_size, 0]
test_features, test_labels = train_data[train_size:, 1:], train_data[train_size:, 0]
total_features, total_labels = train_data[:, 1:], train_data[:, 0]

# Pretreating submit data (normalize & type_conversion)
submit_data = torch.as_tensor(submit_data / 255.0, dtype=torch.float32)

In [ ]:
print(train_features.shape, train_labels.shape)
print(test_features.shape, test_labels.shape)
print(total_features.shape, total_labels.shape)

In [ ]:
# Build custom datasets
class SelfBuildDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.as_tensor(features / 255.0, dtype=torch.float32)
        self.labels = torch.as_tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        X = self.features[idx]
        y = self.labels[idx]
        return X, y

train_dataset = SelfBuildDataset(train_features, train_labels)
test_dataset = SelfBuildDataset(test_features, test_labels)
total_dataset = SelfBuildDataset(total_features, total_labels)

In [ ]:
# Build DataLoader
batch_size = 512
num_loader_workers = 4

train_data_iter = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_loader_workers)
test_data_iter = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_loader_workers)
total_data_iter = DataLoader(total_dataset, batch_size=batch_size, shuffle=True, num_workers=num_loader_workers)

# Design Model Architecture

In [ ]:
# Model Architecture
def build_mlp(layer_dims, activation=None):
    layers = []
    for i in range(len(layer_dims) - 1):
        layers.append(nn.Linear(layer_dims[i], layer_dims[i + 1]))
        if i < (len(layer_dims) - 1) - 1 and activation != None:
            layers.append(activation())
    return nn.Sequential(*layers)

# linear_neural_network = build_mlp([784, 10])
linear_neural_network = build_mlp([784, 2048, 10], activation=nn.ReLU).to(device)

In [ ]:
# Init model parameters
def init_weights(layer):
    if type(layer) == nn.Linear:
        nn.init.normal_(layer.weight, mean=0, std=1e-2)
        nn.init.zeros_(layer.bias)

_ = linear_neural_network.apply(init_weights)

# Train Model

In [ ]:
# Set loss function
cross_entropy = nn.CrossEntropyLoss(reduction='mean')

In [ ]:
# Set optimizer
optimizer = torch.optim.SGD

In [ ]:
# Set hyper-parameters
num_epochs = 400
lr = 1e-2
net = linear_neural_network
loss = cross_entropy
trainer = optimizer(net.parameters(), lr=lr)

In [ ]:
# Start training
def right_predictions_num(y_hat, y):
    y_hat = y_hat.argmax(axis=1)
    cmp = (y_hat.type(y.dtype) == y)
    return float(cmp.type(y.dtype).sum())

def evaluate_by_test_dataset(net, test_data_iter):
    net.eval()   # Set model to evaluating mode
    trace_data = torch.tensor([0, 0, 0]).type(torch.float32)   # sum[loss]; num[right_predict]; num[test_sample];
    with torch.no_grad():
        for X, y in test_data_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            right_num = right_predictions_num(y_hat, y)
            trace_data += torch.tensor([float(l) * y.numel(), right_num, y.numel()])
    return (trace_data[0] / trace_data[2]), (trace_data[1] / trace_data[2])    # average[loss]; acc;

def train_epoch(net, train_data_iter, loss, trainer):
    net.train()    # Set model to training mode
    trace_data = torch.tensor([0, 0, 0]).type(torch.float32)    # sum[loss]; sum[acc]; num[sample];
    for X, y in train_data_iter:
        X, y = X.to(device), y.to(device)
        y_hat = net(X)
        l = loss(y_hat, y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
        trace_data += torch.tensor([float(l) * y.numel(), right_predictions_num(y_hat, y), y.numel()])
    return (trace_data[0] / trace_data[2]), (trace_data[1] / trace_data[2])    # average[loss]; acc;

def train_in_training_dataset(net, train_data_iter, test_data_iter, loss, trainer, num_epochs):
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(net, train_data_iter, loss, trainer)
        test_loss, test_acc = evaluate_by_test_dataset(net, test_data_iter)
        print(f"epoch {epoch + 1:0>2}/{num_epochs}:")
        print("\t" + f"train_loss {train_loss:>10.8f}" + f", train_acc {train_acc:>10.8f}")
        print("\t" + f"test_loss  {test_loss:>10.8f}" +  f", test_acc  {test_acc:>10.8f}")

train_in_training_dataset(net, train_data_iter, test_data_iter, loss, trainer, num_epochs)

In [ ]:
# Use total dataset to train model for submission
def train_in_total_dataset(net, total_data_iter, loss, trainer, num_epochs):
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(net, total_data_iter, loss, trainer)
        print(f"epoch {epoch + 1:0>2}/{num_epochs}: trian_loss {train_loss:>10.8f}, train_acc {train_acc:>10.8f}")

linear_neural_network.apply(init_weights)    # Refresh network parameters for new training in total dataset
train_in_total_dataset(net, total_data_iter, loss, trainer, num_epochs)

# Generate Submission

In [ ]:
# Model prediction
def predict(net, input_tensor):
    net.eval()
    with torch.no_grad():
        rst = net(input_tensor).argmax(axis=1).type(torch.long)
    return rst

prediction_rst = predict(net, submit_data.to(device)).reshape(-1, 1)
print(prediction_rst.shape)

In [ ]:
# Generate idxs and concat it with rst
idxs = torch.arange(1, len(prediction_rst) + 1).reshape(-1, 1).to(device)
prediction_rst = torch.cat((idxs, prediction_rst), 1)
print(prediction_rst.shape)

In [ ]:
# Save result
np_prediction_rst = prediction_rst.cpu().numpy()
df = pd.DataFrame(np_prediction_rst, columns=['ImageId', 'Label'])
df.to_csv('submission.csv', index=False)